In [1]:
import pandas as pd
import numpy as np
CREATE_CONDENSD = True

In [36]:
df = pd.read_csv('../../data/knn_recommendations/company-similarity.csv', dtype={'code': str, 'id': str})
#add in some fake na
df.at[0, 'lat'] = None
df.at[0, 'long'] = None
df.at[0, 'code'] = None
df    

,id,lat,long,code
0,14827600,NaN,NaN,None
1,14829773,51.399510,0.014750,56101
2,14837638,51.511560,0.000000,66190
3,14892324,50.708650,-3.527200,82190
4,14899965,51.530960,0.077450,88990
...,...,...,...,...
7899,15223301,53.479500,-2.242310,26800
7900,15554568,51.538827,-0.259563,10420
7901,00665289,51.512550,0.000000,65201
7902,00010483,53.757150,-1.753610,24310


In [37]:
df.dropna().shape

(7903, 4)

In [4]:
# df['lat'] = df['lat'].fillna(df['lat'].mean())
# df['long'] = df['long'].fillna(df['long'].mean())
# df

,id,lat,long,activity_code
index,,,,
0,UK14774138,50.381220,-4.160850,62012
1,UK14794344,51.429590,-0.547460,82990
2,UK14799610,51.747310,-0.313780,41100
3,UK14804001,55.206640,-6.523650,47290
4,UK14804569,53.086690,-2.444910,53201
...,...,...,...,...
19276723,UK14580535,56.458705,-2.994404,None
19276724,UK14580555,55.950773,-3.205582,None
19276725,UK14580603,53.587020,-0.650410,82990


In [ ]:
# df = pd.get_dummies(data=df, columns=['code'], dummy_na=True)
# df